In [ ]:
# @title **1. Setup Lingkungan & Instalasi**
# Langkah pertama: Siapkan lingkungan Colab kita.

# Cek tipe GPU yang digunakan
!nvidia-smi

# Hubungkan Colab ke Google Drive Anda
from google.colab import drive
drive.mount('/content/drive')

print("✅ Lingkungan berhasil disiapkan!")

Sat Oct 25 05:06:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
print("🔧 Mengupgrade NumPy dan Protobuf...")
!pip install --upgrade numpy protobuf

# 3. Install PyTorch yang kompatibel dengan CUDA di Colab
# Biarkan pip memilih versi terbaru yang cocok dengan NumPy 2.x
print("🔧 Menginstal PyTorch...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

🔧 Mengupgrade NumPy dan Protobuf...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.2/323.2 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 whi

🔧 Menginstal PyTorch...
Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_path = "/content/drive/MyDrive/doublebody/dataset.zip"
extract_path = "/content/drive/MyDrive/doublebody/doublebody_dataset"

In [ ]:
import os
!mkdir -p /content/doublebody_dataset
!unzip -q -o "/content/drive/MyDrive/doublebody/dataset.zip" -d /content/doublebody_dataset


In [ ]:

import yaml
import os

dataset_base_path = '/content/doublebody_dataset'

if not os.path.exists(dataset_base_path):
    raise FileNotFoundError(f"Folder '{dataset_base_path}' tidak ditemukan di Google Drive Anda. Pastikan Anda sudah membuatnya.")

train_path = os.path.join(dataset_base_path, 'train', 'images')
val_path = os.path.join(dataset_base_path, 'valid', 'images')
test_path = os.path.join(dataset_base_path, 'test', 'images')

for path in [train_path, val_path, test_path]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Sub-folder '{path}' tidak ditemukan. Periksa kembali struktur folder Anda.")

data_config = {
    'train': train_path,
    'val': val_path,
    'test': test_path,
    'nc': 1,  # Jumlah kelas (hanya 'person')
    'names': ['person'] # Nama kelas
}

# Path untuk menyimpan file data.yaml
data_yaml_path = os.path.join(dataset_base_path, 'data.yaml')

# Tulis file data.yaml
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f)

print(f"✅ File 'data.yaml' berhasil dibuat di: {data_yaml_path}")
print("Isi file data.yaml:")
print(data_config)

✅ File 'data.yaml' berhasil dibuat di: /content/doublebody_dataset/data.yaml
Isi file data.yaml:
{'train': '/content/doublebody_dataset/train/images', 'val': '/content/doublebody_dataset/valid/images', 'test': '/content/doublebody_dataset/test/images', 'nc': 1, 'names': ['person']}


In [ ]:
from ultralytics import YOLO


data_yaml_path = '/content/doublebody_dataset/data.yaml'

model = YOLO('yolov9s.pt')

results = model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16, # Kurangi batch jika mengalami error CUDA out of memory
    name='yolov9_custom_person',
    project='/content/drive/MyDrive/doublebody/Output' # Ensure this is a string
)

print("Pelatihan selesai! Model tersimpan di Google Drive Anda.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/doublebody_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7

In [ ]:
# @title 6. Load Model Terlatih dan Tes pada Gambar dari Folder Test
import cv2
from google.colab.patches import cv2_imshow
import os

# Path ke model terbaik yang telah dilatih
best_model_path = '/content/drive/MyDrive/YOLOv9_Training_From_Drive/yolov9_custom_person/weights/best.pt'

# Load model terlatih
trained_model = YOLO(best_model_path)

# --- PENTING: GANTI NAMA FILE GAMBAR DI BAWAH INI ---
# Pastikan gambar ini ada di folder 'test/images' di Google Drive Anda
test_image_name = 'gambar_test_1.jpg' # <--- GANTI INI
test_image_path = f'/content/drive/MyDrive/YOLOv9_Dataset/test/images/{test_image_name}'

if not os.path.exists(test_image_path):
    print(f"Error: Gambar uji '{test_image_path}' tidak ditemukan. Pastikan nama file benar.")
else:
    # Jalankan inferensi
    results = trained_model(test_image_path)

    # Visualisasikan hasil
    annotated_image = results[0].plot()

    # Tampilkan gambar
    print(f"Hasil Deteksi pada Gambar: {test_image_name}")
    cv2_imshow(annotated_image)

NameError: name 'YOLO' is not defined

In [ ]:
# @title 7. Fungsi Integrasi YOLOv9 dan MediaPipe
import mediapipe as mp

# Inisialisasi MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False)
mp_drawing = mp.solutions.drawing_utils

def analyze_persons(image_path, yolo_model):
    """
    Fungsi untuk mendeteksi orang, menghitungnya, dan menganalisis pose dengan MediaPipe.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Tidak bisa membaca gambar dari {image_path}")
        return None

    # 1. Deteksi orang dengan YOLOv9
    results = yolo_model(image)
    person_detections = 0

    # Iterasi setiap deteksi
    for result in results:
        boxes = result.boxes
        for box in boxes:
            if int(box.cls) == 0: # Filter hanya kelas 'person'
                person_detections += 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, f'Person {confidence:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # 2. Analisis pose dengan MediaPipe pada area bounding box
                person_crop = image[y1:y2, x1:x2]
                rgb_crop = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
                pose_results = pose.process(rgb_crop)

                # 3. Inferensi orientasi (depan/belakang)
                orientation_text = "Orientasi: Tidak Diketahui"
                if pose_results.pose_landmarks:
                    nose = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
                    left_eye = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE]
                    right_eye = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE]

                    if nose.visibility > 0.5 and left_eye.visibility > 0.5 and right_eye.visibility > 0.5:
                        orientation_text = "Orientasi: Menghadap Depan"
                    else:
                        orientation_text = "Orientasi: Menghadap Belakang/Samping"

                    mp_drawing.draw_landmarks(person_crop, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                cv2.putText(image, orientation_text, (x1, y2 + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

    # 4. Logika Utama: Tampilkan peringatan jika > 1 orang
    if person_detections > 1:
        warning_text = f"WASPADA: Terdeteksi {person_detections} Orang!"
        cv2.putText(image, warning_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 3)
    else:
        status_text = f"Aman: Terdeteksi {person_detections} Orang."
        cv2.putText(image, status_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)

    return image

# Uji fungsi dengan gambar lain dari folder test
# --- GANTI NAMA FILE GAMBAR DI BAWAH INI ---
test_image_multi_name = 'gambar_test_lainnya.jpg' # <--- GANTI INI
test_image_multi_path = f'/content/drive/MyDrive/YOLOv9_Dataset/test/images/{test_image_multi_name}'

if os.path.exists(test_image_multi_path):
    final_image = analyze_persons(test_image_multi_path, trained_model)
    if final_image is not None:
        print("Hasil Analisis Final:")
        cv2_imshow(final_image)
else:
    print(f"Error: Gambar uji '{test_image_multi_path}' tidak ditemukan. Lewati langkah ini atau perbaiki path.")